In [1]:
# Imports
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import numpy as np
import os
import tensorflow as tf
import sys
import cv2
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import categorical_crossentropy
# Set batch size for training and validation
batch_size = 32

In [2]:
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [3]:
model_dictionary

{'DenseNet121': <function keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>,
 'DenseNet169': <function keras.applications.densenet.DenseNet169(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>,
 'DenseNet201': <function keras.applications.densenet.DenseNet201(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>,
 'EfficientNetB0': <function keras.applications.efficientnet.EfficientNetB0(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>,
 'EfficientNetB1': <function keras.applications.efficientnet.EfficientNetB1(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None,

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip "/content/gdrive/MyDrive/Data/archive.zip"

Archive:  /content/gdrive/MyDrive/Data/archive.zip
replace animals/animals/antelope/02f4b3be2d.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace animals/animals/antelope/03d7fc0888.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
data_dir = '/content/animals/animals'
Name = os.listdir(data_dir)
#Name.sort()
print(Name)
print(len(Name))

['crow', 'tiger', 'rhinoceros', 'kangaroo', 'seahorse', 'bee', 'possum', 'hornbill', 'badger', 'horse', 'octopus', 'boar', 'ox', 'wombat', 'fox', 'donkey', 'koala', 'crab', 'sandpiper', 'okapi', 'dragonfly', 'lion', 'orangutan', 'owl', 'woodpecker', 'bear', 'eagle', 'hyena', 'hare', 'oyster', 'turkey', 'pig', 'hamster', 'caterpillar', 'lobster', 'sheep', 'hedgehog', 'wolf', 'seal', 'snake', 'cockroach', 'goat', 'lizard', 'porcupine', 'penguin', 'raccoon', 'otter', 'reindeer', 'gorilla', 'butterfly', 'squid', 'duck', 'rat', 'chimpanzee', 'jellyfish', 'squirrel', 'beetle', 'leopard', 'pelecaniformes', 'bison', 'swan', 'hippopotamus', 'flamingo', 'grasshopper', 'panda', 'shark', 'cat', 'dog', 'sparrow', 'mouse', 'goose', 'pigeon', 'moth', 'hummingbird', 'cow', 'whale', 'zebra', 'deer', 'antelope', 'goldfish', 'turtle', 'mosquito', 'coyote', 'dolphin', 'bat', 'elephant', 'parrot', 'starfish', 'fly', 'ladybugs']
90


In [ ]:
import os
import numpy as np
import shutil

rootdir= '/content/animals/'
classes = Name

for i in classes:
  os.makedirs(rootdir +'/train/' + i)
  os.makedirs(rootdir +'/test/' + i)
  source = '/content/animals/animals/' + i
  allFileNames = os.listdir(source)
  np.random.shuffle(allFileNames)

  test_ratio = 0.05
  
  train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                      [int(len(allFileNames)* (1 - test_ratio))])

  train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

  for name in train_FileNames:
    shutil.copy(name, rootdir +'/train/' + i)

  for name in test_FileNames:
    shutil.copy(name, rootdir +'/test/' + i)

In [ ]:
# import shutil
# shutil.rmtree('/content/animals/test')
# shutil.rmtree('/content/animals/train')

In [ ]:
train_dir = '/content/animals/train'
train_processed_224  = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 32,
                                                                    image_size = (224,224),
                                                                    shuffle = True, 
                                                                    seed = 505,
                                                                    validation_split=0.1,
                                                                    subset = "training")
validation_processed_224 = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 32,
                                                                    image_size = (224,224),
                                                                    shuffle = True, 
                                                                    seed = 505,
                                                                    validation_split=0.1,
                                                                    subset = "validation")

train_processed_331  = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 32,
                                                                    image_size = (331,331),
                                                                    shuffle = True, 
                                                                    seed = 505,
                                                                    validation_split=0.1,
                                                                    subset = "training")
validation_processed_331 = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 32,
                                                                    image_size = (331,331),
                                                                    shuffle = True, 
                                                                    seed = 505,
                                                                    validation_split=0.1,
                                                                    subset = "validation")

Found 5130 files belonging to 90 classes.
Using 4617 files for training.
Found 5130 files belonging to 90 classes.
Using 513 files for validation.
Found 5130 files belonging to 90 classes.
Using 4617 files for training.
Found 5130 files belonging to 90 classes.
Using 513 files for validation.


In [ ]:
# (train, validation), metadata = tfds.load('cats_vs_dogs', split=['train[:70%]', 'train[70%:]'], with_info=True, as_supervised=True)

# Number of training examples and labels
num_train = len(train_processed_224)
num_validation = len(validation_processed_224)
num_classes = len(Name)
num_iterations = int(num_train/batch_size)

# # Print important info
# print(f'Num train images: {num_train} \
#         \nNum validation images: {num_validation} \
#         \nNum classes: {num_classes} \
#         \nNum iterations per epoch: {num_iterations}')

In [ ]:
def normalize_img(image, label, img_size):
    # Resize image to the desired img_size and normalize it
    # One hot encode the label
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255
    label = tf.one_hot(label, depth=num_classes)
    return image, label
    
def preprocess_data(train, validation, batch_size, img_size):
    # Apply the normalize_img function on all train and validation data and create batches
    train_processed = train.map(lambda image, label: normalize_img(image, label, img_size))
    
    # If your data is already batched (eg, when using the image_dataset_from_directory function), remove .batch(batch_size)
    train_processed = train_processed.batch(batch_size).repeat()
    
    validation_processed = validation.map(lambda image, label: normalize_img(image, label, img_size))
    
    # If your data is already batched (eg, when using the image_dataset_from_directory function), remove .batch(batch_size)
    validation_processed = validation_processed.batch(batch_size)
    
    return train_processed, validation_processed
    
# Run preprocessing
train_processed_224, validation_processed_224 = preprocess_data(train_dataset, validation_dataset, batch_size, img_size=[224,224])
train_processed_331, validation_processed_331 = preprocess_data(train_dataset, validation_dataset, batch_size, img_size=[331,331])

In [ ]:
# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}

for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_processed = train_processed_331
        validation_processed = validation_processed_331
    else:
        input_shape=(224,224,3)
        train_processed = train_processed_224
        validation_processed = validation_processed_224
        
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    pre_trained_model = model(include_top=False, pooling='avg', input_shape=input_shape)
    pre_trained_model.trainable = False
    
    # custom modifications on top of pre-trained model and fit
    clf_model = tf.keras.models.Sequential()
    clf_model.add(pre_trained_model)
    # clf_model.add(tf.keras.layers.BatchNormalization())
    # clf_model.add(Dropout(rate=.45))
    clf_model.add(tf.keras.layers.Dense(90, activation='softmax'))
    clf_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = clf_model.fit(train_processed, epochs=3, validation_data=validation_processed, steps_per_epoch=num_iterations)

    # Calculate all relevant metrics
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])

  0%|          | 0/35 [00:00<?, ?it/s]

Epoch 1/3
4/4 [==============================] - 18s 3s/step - loss: 16.7677 - accuracy: 0.0156 - val_loss: 12.9258 - val_accuracy: 0.0156
Epoch 2/3
4/4 [==============================] - 6s 2s/step - loss: 12.1681 - accuracy: 0.0000e+00 - val_loss: 10.9227 - val_accuracy: 0.0156
Epoch 3/3
4/4 [==============================] - 11s 4s/step - loss: 9.9494 - accuracy: 0.0234 - val_loss: 9.6820 - val_accuracy: 0.0117


  3%|▎         | 1/35 [00:55<31:43, 55.99s/it]

Epoch 1/3
4/4 [==============================] - 24s 3s/step - loss: 20.6687 - accuracy: 0.0234 - val_loss: 16.7132 - val_accuracy: 0.0175
Epoch 2/3
4/4 [==============================] - 7s 2s/step - loss: 18.6038 - accuracy: 0.0156 - val_loss: 14.6224 - val_accuracy: 0.0136
Epoch 3/3
4/4 [==============================] - 7s 2s/step - loss: 13.7843 - accuracy: 0.0156 - val_loss: 12.1860 - val_accuracy: 0.0156


  6%|▌         | 2/35 [01:55<31:57, 58.10s/it]

...

 94%|█████████▍| 33/35 [33:41<01:30, 45.38s/it]

80150528/80134624 [==============================] - 1s 0us/step
Epoch 1/3
4/4 [==============================] - 11s 3s/step - loss: 18.3094 - accuracy: 0.0078 - val_loss: 17.2878 - val_accuracy: 0.0175
Epoch 2/3
4/4 [==============================] - 9s 3s/step - loss: 14.9997 - accuracy: 0.0312 - val_loss: 14.8619 - val_accuracy: 0.0331
Epoch 3/3
4/4 [==============================] - 8s 3s/step - loss: 14.7785 - accuracy: 0.0156 - val_loss: 13.6542 - val_accuracy: 0.0370


 97%|█████████▋| 34/35 [34:25<00:44, 44.77s/it]

83697664/83683744 [==============================] - 1s 0us/step
Epoch 1/3
4/4 [==============================] - 18s 3s/step - loss: 52.2729 - accuracy: 0.0078 - val_loss: 36.1858 - val_accuracy: 0.0136
Epoch 2/3
4/4 [==============================] - 8s 2s/step - loss: 33.8044 - accuracy: 0.0000e+00 - val_loss: 28.4736 - val_accuracy: 0.0156
Epoch 3/3
4/4 [==============================] - 8s 3s/step - loss: 30.0784 - accuracy: 0.0078 - val_loss: 23.5044 - val_accuracy: 0.0117


100%|██████████| 35/35 [35:01<00:00, 60.05s/it]


In [ ]:
# Convert Results to DataFrame for easy viewing
benchmark_df = pd.DataFrame(model_benchmarks)

# sort in ascending order of num_model_params column
benchmark_df.sort_values('val_loss', inplace=True, ascending=False)

benchmark_df

,model_name,num_model_params,validation_accuracy
10,EfficientNetB7,64097687,0.575049
9,EfficientNetB6,40960143,0.561404
8,EfficientNetB5,28513527,0.526316
15,EfficientNetV2L,117746848,0.522417
16,EfficientNetV2M,53150388,0.512671
17,EfficientNetV2S,20331360,0.510721
14,EfficientNetV2B3,12930622,0.502924
13,EfficientNetV2B2,8769374,0.475634
12,EfficientNetV2B1,6931124,0.438596
6,EfficientNetB3,10783535,0.434698
